# AutoGen - Smart Health Asistant
#### References
#### https://microsoft.github.io/autogen/0.2/docs/tutorial/introduction
#### https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns#sequential-chats
#### https://microsoft.github.io/autogen/0.2/docs/tutorial/tool-use

In [5]:
# pip install pyautogen
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from autogen import (
    AssistantAgent,
    UserProxyAgent,
)
from IPython.display import Markdown, display
import os
import numpy as np
import json

C:\Users\krish\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\krish\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\krish\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [7]:
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0.1,
}

openai_api_key = "<API_KEY>"
os.environ["OPENAI_API_KEY"] = openai_api_key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [23]:
def calculate_bmi(weight_kg: float, height_cm: float) -> float:
    """
    Calculate BMI using weight (kg) and height (cm).
    """
    height_m = height_cm / 100
    return round(weight_kg / (height_m ** 2), 2)

In [55]:
user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
    system_message=(
        "You are the User Proxy Agent. "
        "Collect user health inputs once and share them with other agents. "
        "Drive the sequential conversation flow."
    ),
    is_termination_msg=lambda msg: (
        isinstance(msg.get("content"), str)
        and "<BMI_DONE>" in msg["content"]
    ),
)

bmi_agent = AssistantAgent(
    name="BMIAgent",
    llm_config=llm_config,
    system_message=(
        "You are the BMI Agent. "
        "Using the provided user data, call the calculate_bmi tool, "
        "classify the BMI, and provide health recommendations. "
        "After calculating the BMI and providing health recommendations, end your response with <BMI_DONE>."
    ),
)

diet_planner_agent = AssistantAgent(
    name="DietPlannerAgent",
    llm_config=llm_config,
    system_message=(
        "You are the Diet Planner Agent. "
        "Use BMI insights and dietary preferences to suggest a tailored meal plan."
    ),
)

workout_scheduler_agent = AssistantAgent(
    name="WorkoutSchedulerAgent",
    llm_config=llm_config,
    system_message=(
        "You are the Workout Scheduler Agent. "
        "Create a weekly workout plan using diet recommendations, age, and gender. "
        "Return the final output in well-structured Markdown with headings and bullet points."
    ),
)

In [57]:
bmi_agent.register_for_llm(
    name="calculate_bmi",
    description="Calculate BMI from weight (kg) and height (cm).",
)(calculate_bmi)

user_proxy.register_for_execution(
    name="calculate_bmi"
)(calculate_bmi)

In [45]:
user_data_prompt = """
Collect the following user health details and store them clearly:

- Weight (kg): 78
- Height (cm): 172
- Age: 29
- Gender: Male
- Dietary Preference: Vegetarian

Confirm once collected.
"""

In [69]:
chat_result = user_proxy.initiate_chat(
    bmi_agent,
    message=user_data_prompt,
    max_turns=5
)

UserProxy (to BMIAgent):


Collect the following user health details and store them clearly:

- Weight (kg): 78
- Height (cm): 172
- Age: 29
- Gender: Male
- Dietary Preference: Vegetarian

Confirm once collected.


--------------------------------------------------------------------------------
BMIAgent (to UserProxy):

User health details collected:

- Weight: 78 kg
- Height: 172 cm
- Age: 29
- Gender: Male
- Dietary Preference: Vegetarian

Confirmation complete.

--------------------------------------------------------------------------------
UserProxy (to BMIAgent):



--------------------------------------------------------------------------------
BMIAgent (to UserProxy):

***** Suggested tool call (call_1qMmRTDLmnYfCuIl3UKXu9Mn): calculate_bmi *****
Arguments: 
{"weight_kg":78,"height_cm":172}
******************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION

In [75]:
from IPython.display import Markdown, display

final_text = None

for msg in reversed(chat_result.chat_history):
    content = msg.get("content")
    if isinstance(content, str) and content.strip():
        final_text = content
        break

if final_text:
    final_text = final_text.replace("<BMI_DONE>", "").strip()

if final_text:
    display(Markdown(final_text))
else:
    print("No final message found.")

The calculated BMI is 26.37.

### BMI Classification:
- **BMI Range**: 25.0 - 29.9
- **Classification**: Overweight

### Health Recommendations:
1. **Dietary Adjustments**: As a vegetarian, focus on incorporating a variety of whole grains, legumes, fruits, and vegetables. Monitor portion sizes and consider reducing high-calorie foods.
2. **Physical Activity**: Aim for at least 150 minutes of moderate aerobic activity or 75 minutes of vigorous activity each week. Include strength training exercises at least twice a week.
3. **Hydration**: Drink plenty of water throughout the day to stay hydrated.
4. **Regular Check-ups**: Schedule regular health check-ups to monitor your weight and overall health.
5. **Mindful Eating**: Practice mindful eating by paying attention to hunger cues and avoiding distractions during meals.

Consider consulting with a healthcare provider or a nutritionist for personalized advice.

In [85]:
chat_results = user_proxy.initiate_chats(
    [
        {
            "recipient": bmi_agent,
            "message": (
                "User details:\n"
                "- Weight (kg): 78\n"
                "- Height (cm): 172\n"
                "- Age: 29\n"
                "- Gender: Male\n"
                "- Dietary Preference: Vegetarian\n\n"
                "Calculate BMI and provide health recommendations."
            ),
            "max_turns": 2,
            "summary_method": "last_msg",
        },

        {
            "recipient": diet_planner_agent,
            "message": "Use the BMI insights above and dietary preference to create a meal plan.",
            "max_turns": 2,
            "summary_method": "last_msg",
        },

        {
            "recipient": workout_scheduler_agent,
            "message": (
                "Using the BMI insights, meal plan, age (29), and gender (Male), "
                "create a weekly workout schedule."
            ),
            "max_turns": 2,
            "summary_method": "last_msg",
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
UserProxy (to BMIAgent):

User details:
- Weight (kg): 78
- Height (cm): 172
- Age: 29
- Gender: Male
- Dietary Preference: Vegetarian

Calculate BMI and provide health recommendations.

--------------------------------------------------------------------------------
BMIAgent (to UserProxy):

***** Suggested tool call (call_0NdU5QbSN76ZjEQRY4rPa4Nl): calculate_bmi *****
Arguments: 
{"weight_kg":78,"height_cm":172}
******************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculate_bmi...
Call ID: call_0NdU5QbSN76ZjEQRY4rPa4Nl
Input arguments: {'weight_kg': 78, 'height_cm': 172}
UserProxy (to BMIAgent):

***** Response from calling tool (call_0NdU5QbSN76ZjEQRY4rPa4Nl) ****

In [87]:
chat_results

[ChatResult(chat_id=None, chat_history=[{'content': 'User details:\n- Weight (kg): 78\n- Height (cm): 172\n- Age: 29\n- Gender: Male\n- Dietary Preference: Vegetarian\n\nCalculate BMI and provide health recommendations.', 'role': 'assistant', 'name': 'UserProxy'}, {'tool_calls': [{'id': 'call_0NdU5QbSN76ZjEQRY4rPa4Nl', 'function': {'arguments': '{"weight_kg":78,"height_cm":172}', 'name': 'calculate_bmi'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': '26.37', 'tool_responses': [{'tool_call_id': 'call_0NdU5QbSN76ZjEQRY4rPa4Nl', 'role': 'tool', 'content': '26.37'}], 'role': 'tool', 'name': 'UserProxy'}, {'content': 'Your BMI is 26.37, which classifies you as overweight according to the BMI classification. Here are some health recommendations for you:\n\n1. **Dietary Adjustments**: As a vegetarian, focus on incorporating a variety of whole foods such as fruits, vegetables, whole grains, legumes, nuts, and seeds. Ensure you are getting enough protein from sources

In [93]:
def extract_longest_message_per_agent(chat_results):
    results = []

    for chat_result in chat_results:
        agent_messages = {}

        for msg in chat_result.chat_history:
            name = msg.get("name")
            content = msg.get("content")

            if not isinstance(content, str):
                continue

            content = content.strip()
            if not content:
                continue

            if name not in agent_messages or len(content) > len(agent_messages[name]):
                agent_messages[name] = content

        for name, content in agent_messages.items():
            results.append((name, content))

    return results

In [95]:
outputs = extract_longest_message_per_agent(chat_results)

for agent_name, content in outputs:
    display(Markdown(f"""
## 🧠 {agent_name}

{content}
"""))


## 🧠 UserProxy

User details:
- Weight (kg): 78
- Height (cm): 172
- Age: 29
- Gender: Male
- Dietary Preference: Vegetarian

Calculate BMI and provide health recommendations.



## 🧠 BMIAgent

Your BMI is 26.37, which classifies you as overweight according to the BMI classification. Here are some health recommendations for you:

1. **Dietary Adjustments**: As a vegetarian, focus on incorporating a variety of whole foods such as fruits, vegetables, whole grains, legumes, nuts, and seeds. Ensure you are getting enough protein from sources like beans, lentils, tofu, and quinoa.

2. **Portion Control**: Be mindful of portion sizes, especially with high-calorie vegetarian foods like nuts and oils.

3. **Regular Exercise**: Aim for at least 150 minutes of moderate aerobic activity or 75 minutes of vigorous activity each week. Include strength training exercises at least twice a week.

4. **Stay Hydrated**: Drink plenty of water throughout the day, which can help with weight management and overall health.

5. **Monitor Progress**: Keep track of your weight and dietary habits to identify patterns and make adjustments as needed.

6. **Consult a Professional**: Consider speaking with a registered dietitian or nutritionist who can provide personalized advice based on your dietary preferences and health goals.

Making these changes can help you achieve a healthier weight and improve your overall well-being. 

<BMI_DONE>



## 🧠 UserProxy

Use the BMI insights above and dietary preference to create a meal plan.
Context: 
Your BMI is 26.37, which classifies you as overweight according to the BMI classification. Here are some health recommendations for you:

1. **Dietary Adjustments**: As a vegetarian, focus on incorporating a variety of whole foods such as fruits, vegetables, whole grains, legumes, nuts, and seeds. Ensure you are getting enough protein from sources like beans, lentils, tofu, and quinoa.

2. **Portion Control**: Be mindful of portion sizes, especially with high-calorie vegetarian foods like nuts and oils.

3. **Regular Exercise**: Aim for at least 150 minutes of moderate aerobic activity or 75 minutes of vigorous activity each week. Include strength training exercises at least twice a week.

4. **Stay Hydrated**: Drink plenty of water throughout the day, which can help with weight management and overall health.

5. **Monitor Progress**: Keep track of your weight and dietary habits to identify patterns and make adjustments as needed.

6. **Consult a Professional**: Consider speaking with a registered dietitian or nutritionist who can provide personalized advice based on your dietary preferences and health goals.

Making these changes can help you achieve a healthier weight and improve your overall well-being. 

<BMI_DONE>



## 🧠 DietPlannerAgent

Based on your BMI classification and dietary preferences as a vegetarian, here’s a tailored meal plan for a week that focuses on whole foods, portion control, and balanced nutrition. This plan emphasizes protein-rich foods, plenty of fruits and vegetables, and healthy fats while keeping calorie intake in check.

### Weekly Vegetarian Meal Plan

#### Day 1
- **Breakfast**: Overnight oats with almond milk, chia seeds, sliced banana, and a sprinkle of cinnamon.
- **Snack**: A small apple with a tablespoon of almond butter.
- **Lunch**: Quinoa salad with black beans, corn, diced bell peppers, avocado, and lime dressing.
- **Snack**: Carrot sticks with hummus.
- **Dinner**: Stir-fried tofu with mixed vegetables (broccoli, bell peppers, carrots) served over brown rice.

#### Day 2
- **Breakfast**: Smoothie with spinach, banana, protein powder, and almond milk.
- **Snack**: A handful of mixed nuts (watch portion size).
- **Lunch**: Lentil soup with a side of whole-grain bread.
- **Snack**: Greek yogurt with a drizzle of honey and berries.
- **Dinner**: Vegetable curry with chickpeas served with quinoa.

#### Day 3
- **Breakfast**: Whole grain toast with avocado and a sprinkle of sesame seeds.
- **Snack**: Celery sticks with peanut butter.
- **Lunch**: Spinach salad with roasted sweet potatoes, walnuts, feta cheese, and balsamic vinaigrette.
- **Snack**: A pear.
- **Dinner**: Stuffed bell peppers with brown rice, black beans, corn, and spices.

#### Day 4
- **Breakfast**: Chia pudding made with almond milk, topped with mixed berries.
- **Snack**: Cucumber slices with tzatziki.
- **Lunch**: Whole grain wrap with hummus, mixed greens, shredded carrots, and cucumber.
- **Snack**: A small handful of trail mix.
- **Dinner**: Zucchini noodles with marinara sauce and lentil meatballs.

#### Day 5
- **Breakfast**: Smoothie bowl with blended banana, spinach, and almond milk, topped with granola and sliced kiwi.
- **Snack**: A small orange.
- **Lunch**: Farro salad with roasted vegetables and a lemon-tahini dressing.
- **Snack**: Bell pepper slices with guacamole.
- **Dinner**: Vegetable stir-fry with tempeh served over quinoa.

#### Day 6
- **Breakfast**: Oatmeal topped with sliced almonds and fresh berries.
- **Snack**: A banana.
- **Lunch**: Chickpea salad with cucumbers, tomatoes, parsley, and lemon dressing.
- **Snack**: Air-popped popcorn (lightly salted).
- **Dinner**: Eggplant Parmesan with a side of steamed broccoli.

#### Day 7
- **Breakfast**: Smoothie with kale, banana, and almond milk.
- **Snack**: A small handful of walnuts.
- **Lunch**: Vegetable sushi rolls with avocado, cucumber, and carrots.
- **Snack**: Cherry tomatoes with mozzarella balls.
- **Dinner**: Cauliflower tacos with black beans, avocado, and salsa.

### Additional Tips:
- **Hydration**: Aim for at least 8 glasses of water daily.
- **Portion Control**: Use smaller plates to help manage portion sizes.
- **Exercise**: Incorporate a mix of cardio and strength training into your routine.
- **Meal Prep**: Consider preparing meals in advance to save time and ensure you stick to your plan.

Feel free to adjust the meal plan according to your taste preferences and any specific dietary needs. Enjoy your meals and the journey towards a healthier lifestyle!



## 🧠 UserProxy

Using the BMI insights, meal plan, age (29), and gender (Male), create a weekly workout schedule.
Context: 
Your BMI is 26.37, which classifies you as overweight according to the BMI classification. Here are some health recommendations for you:

1. **Dietary Adjustments**: As a vegetarian, focus on incorporating a variety of whole foods such as fruits, vegetables, whole grains, legumes, nuts, and seeds. Ensure you are getting enough protein from sources like beans, lentils, tofu, and quinoa.

2. **Portion Control**: Be mindful of portion sizes, especially with high-calorie vegetarian foods like nuts and oils.

3. **Regular Exercise**: Aim for at least 150 minutes of moderate aerobic activity or 75 minutes of vigorous activity each week. Include strength training exercises at least twice a week.

4. **Stay Hydrated**: Drink plenty of water throughout the day, which can help with weight management and overall health.

5. **Monitor Progress**: Keep track of your weight and dietary habits to identify patterns and make adjustments as needed.

6. **Consult a Professional**: Consider speaking with a registered dietitian or nutritionist who can provide personalized advice based on your dietary preferences and health goals.

Making these changes can help you achieve a healthier weight and improve your overall well-being. 

<BMI_DONE>
It seems like your message was cut off. If you have any questions or need further assistance with your meal plan or dietary preferences, feel free to let me know! I'm here to help.



## 🧠 WorkoutSchedulerAgent

# Weekly Workout Schedule for a 29-Year-Old Male (BMI: 26.37)

## Overview
This workout plan is designed to help you achieve a healthier weight while incorporating your vegetarian diet. The schedule includes a mix of aerobic activities, strength training, and flexibility exercises.

## Weekly Workout Plan

### Monday
- **Activity**: Moderate Aerobic Exercise
  - **Duration**: 30 minutes
  - **Type**: Brisk walking or cycling
- **Strength Training**: Upper Body
  - **Exercises**: 
    - Push-ups: 3 sets of 10-15 reps
    - Dumbbell shoulder press: 3 sets of 10-12 reps
    - Bent-over rows: 3 sets of 10-12 reps

### Tuesday
- **Activity**: Flexibility and Core
  - **Duration**: 30 minutes
  - **Type**: Yoga or Pilates
- **Focus**: Stretching major muscle groups and core strengthening exercises.

### Wednesday
- **Activity**: Vigorous Aerobic Exercise
  - **Duration**: 30 minutes
  - **Type**: Running or high-intensity interval training (HIIT)
  
### Thursday
- **Activity**: Strength Training
  - **Exercises**: 
    - Squats: 3 sets of 10-15 reps
    - Lunges: 3 sets of 10-12 reps (each leg)
    - Plank: 3 sets of 30-60 seconds

### Friday
- **Activity**: Moderate Aerobic Exercise
  - **Duration**: 30 minutes
  - **Type**: Swimming or cycling

### Saturday
- **Activity**: Active Recovery
  - **Duration**: 30-45 minutes
  - **Type**: Light walking or recreational activities (e.g., hiking, playing a sport)

### Sunday
- **Rest Day**
  - Focus on hydration and meal prep for the upcoming week.

## Dietary Recommendations
- **Protein Sources**: Incorporate beans, lentils, tofu, and quinoa into your meals.
- **Fruits and Vegetables**: Aim for a variety of colors and types to ensure a range of nutrients.
- **Whole Grains**: Include brown rice, whole wheat bread, and oats.
- **Healthy Fats**: Use nuts and seeds in moderation; consider avocados and olive oil.
- **Hydration**: Drink at least 8-10 glasses of water daily.

## Monitoring Progress
- **Track**: Keep a journal of your workouts and meals.
- **Adjust**: Make changes based on your progress and how you feel.

## Consultation
- **Professional Guidance**: Consider consulting a registered dietitian for personalized advice tailored to your dietary preferences and health goals.

---

By following this structured workout plan and dietary recommendations, you can work towards achieving a healthier weight and improving your overall well-being. Stay consistent and listen to your body!
